# i-Eye Saffron Image Classifier

In [2]:
from keras.models import load_model
import ipywidgets as widgets
import io
from PIL import Image
import numpy as np
from keras.preprocessing.image import img_to_array
from IPython import display
import whatimage
import pyheif

In [3]:
model = load_model('vgg_model_2.hdf5')

In [4]:
def show_my_image():
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content'])) 
    #return the image 
    return img.resize((400,400))

In [5]:
def decodeImage(bytesIo):
    fmt = whatimage.identify_image(bytesIo)
    if fmt in ['heic','heif']:
        i = pyheif.read_heif(bytesIo)
        pi = Image.frombytes(mode=i.mode, size=i.size, data=i.data)
    else:
        pi = Image.open(io.BytesIO(bytesIo))
    return pi

In [6]:
def predict(image: Image.Image):
    for name, file_info in btn_upload.value.items():
        img = decodeImage((file_info['content']))
    img = img_to_array(img.resize(size=(224,224)))
    img = np.expand_dims(img, axis = 0)
    img = img / 127.5 - 1.0    
    result = model.predict(img)    
    if result[0][1] < 0.9 :
        lbl_pred.value = "Predicted Class : " + "Non-Saffron"
    else:
        lbl_pred.value = "Predicted Class : " + "Saffron"
    saffron_prob.value = "Probability of Saffron " + f"{result[0][1]*100:0.2f}%"  
    nsaffron_prob.value = "Probability of Non-Saffron " + f"{result[0][0]*100:0.2f}%"
    # clear the output
    out_pl.clear_output()
     # display the uploaded image
    with out_pl:
        display.display(show_my_image())

In [7]:
# Elements Of the App
# upload Button
btn_upload = widgets.FileUpload()
# output widget to display image 
out_pl = widgets.Output()
# label for the result of prediction
lbl_pred = widgets.Label()
saffron_prob = widgets.Label()
nsaffron_prob = widgets.Label()
# Button to classify the uploaded image 
btn_run = widgets.Button(description='Classify')
# event for classify button to predict the result and display uploaded image
btn_run.on_click(predict)

In [8]:
widgets.VBox([widgets.Label('Select Image'),btn_upload,btn_run,out_pl,lbl_pred,saffron_prob,nsaffron_prob],box_style='info')

VBox(box_style='info', children=(Label(value='Select Image'), FileUpload(value={}, description='Upload'), Butt…